In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1000

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [02:15, 9403.19it/s]
  2%|▏         | 22133/1271460 [00:00<00:05, 221252.14it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:04<00:00, 256223.11it/s]


In [5]:
gap = 100
add_length = 20

def split_sentence(data, k):
    
    start_index = max(0, k*gap - add_length)
    end_index = min(add_length + (k+1)*gap, max_sequence_length_api)
        
    return data[:, start_index:end_index, :]

def get_target_tensor(data, k):
    
    if k == 0:
        return data[:, :gap, :]
    
    if k == max_sequence_length_api/gap:
        return data[:, add_length:, :]
    
    else:
        return data[:, add_length:(gap+add_length), :]

def attention(x):
    
    q1, q2 = x[0], x[1]
    
    #compute match matrix euclidean match score
    match_matrix = 1. + K.sqrt(
        -2 * K.batch_dot(q1, q2, axes=[2, 2]) +
        K.expand_dims(K.sum(K.square(q1), axis=2), 2) +
        K.expand_dims(K.sum(K.square(q2), axis=2), 1)
    )
    
    match_matrix = K.maximum(match_matrix, K.epsilon())
    match_matrix = 1. / match_matrix
    #compute attention output
    q1_new = K.batch_dot(K.softmax(match_matrix, axis = 1), q2)
    match_matrix_T = Permute((2, 1))(match_matrix)
    q2_new = K.batch_dot(K.softmax(match_matrix_T, axis = 1), q1)
    
    return [q1_new, q2_new]

In [15]:
def DR_Stack_RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    
    pooled_rnn_max_1 = []
    pooled_rnn_avg_1 = []
    pooled_rnn_max_2 = []
    pooled_rnn_avg_2 = []
    
    att_pooled_rnn_max_1 = []
    att_pooled_rnn_max_2 = []
    att_pooled_rnn_avg_1 = []
    att_pooled_rnn_avg_2 = []
    
    for k_ in range(10):
        
        temp = Lambda(split_sentence, name = 'split_sentence'+str(k_), arguments={'k':k_})(char_embedding)
        
        temp = BatchNormalization()(temp)
        
        char_rnn_1= char_rnn_layer_1(temp)
        char_rnn_2= char_rnn_layer_2(char_rnn_1)
        
        
        #max and  avg for temp and outputs of rnn 
        char_rnn_maxpooling_1 = GlobalMaxPooling1D()(Lambda(get_target_tensor,arguments={'k':k_})(char_rnn_1))
        char_rnn_avgpooling_1 = GlobalAveragePooling1D()(Lambda(get_target_tensor, arguments={'k':k_})(char_rnn_1))
        
        char_rnn_maxpooling_2 = GlobalMaxPooling1D()(Lambda(get_target_tensor, arguments={'k':k_})(char_rnn_2))
        char_rnn_avgpooling_2 = GlobalAveragePooling1D()(Lambda(get_target_tensor, arguments={'k':k_})(char_rnn_2))
        
        pooled_rnn_max_1.append(char_rnn_maxpooling_1)
        pooled_rnn_max_2.append(char_rnn_maxpooling_2)
        
        pooled_rnn_avg_1.append(char_rnn_avgpooling_1)
        pooled_rnn_avg_2.append(char_rnn_avgpooling_2)
        
        #max and avg for temp and outputs of rnn after attention
        #batch normalization
        temp = BatchNormalization()(temp)
        char_rnn_2 = BatchNormalization()(char_rnn_2)
        
        r = Lambda(attention)([temp, char_rnn_2])
    
        attention_map_left = r[0]
        attention_map_right = r[1]
        
        attention_map_left = Lambda(get_target_tensor,arguments={'k':k_})(attention_map_left)
        attention_map_right = Lambda(get_target_tensor,arguments={'k':k_})(attention_map_right)
        
        att_maxpooling_left = GlobalMaxPooling1D()(attention_map_left)
        att_maxpooling_right = GlobalMaxPooling1D()(attention_map_right)

        att_avgpooling_left = GlobalAveragePooling1D()(attention_map_left)
        att_avgpooling_right = GlobalAveragePooling1D()(attention_map_right)
        
        att_pooled_rnn_max_1.append(att_maxpooling_left)
        att_pooled_rnn_max_2.append(att_maxpooling_right)
        
        att_pooled_rnn_avg_1.append(att_avgpooling_left)
        att_pooled_rnn_avg_2.append(att_avgpooling_right)
        
    merged_pooled_rnn_max_1 = Concatenate(axis=-1)(pooled_rnn_max_1)
    merged_pooled_rnn_max_1_sigmoid = Dense(100, activation = 'sigmoid')(merged_pooled_rnn_max_1)
    merged_pooled_rnn_max_1_tanh = Dense(100, activation = 'tanh')(merged_pooled_rnn_max_1)

    merged_pooled_rnn_max_2 = Concatenate(axis=-1)(pooled_rnn_max_2)
    merged_pooled_rnn_max_2_sigmoid = Dense(100, activation = 'sigmoid')(merged_pooled_rnn_max_2)
    merged_pooled_rnn_max_2_tanh = Dense(100, activation = 'tanh')(merged_pooled_rnn_max_2)

    merged_pooled_rnn_avg_1 = Concatenate(axis=-1)(pooled_rnn_avg_1)
    merged_pooled_rnn_avg_1_sigmoid = Dense(100, activation = 'sigmoid')(merged_pooled_rnn_avg_1)
    merged_pooled_rnn_avg_1_tanh = Dense(100, activation = 'tanh')(merged_pooled_rnn_avg_1)
    
    merged_pooled_rnn_avg_2 = Concatenate(axis=-1)(pooled_rnn_avg_2)
    merged_pooled_rnn_avg_2_sigmoid = Dense(100, activation = 'sigmoid')(merged_pooled_rnn_avg_2)
    merged_pooled_rnn_avg_2_tanh = Dense(100, activation = 'tanh')(merged_pooled_rnn_avg_2)
    
    #sub and multi
    max_sub_char_abs_sigmoid = Lambda(lambda x:K.abs(x[0] - x[1]))([merged_pooled_rnn_max_1_sigmoid, merged_pooled_rnn_max_2_sigmoid])
    max_multi_char_sigmoid = Lambda(lambda x: x[0] * x[1])([merged_pooled_rnn_max_1_sigmoid, merged_pooled_rnn_max_2_sigmoid])

    avg_sub_char_abs_sigmoid = Lambda(lambda x:K.abs(x[0] - x[1]))([merged_pooled_rnn_avg_1_sigmoid, merged_pooled_rnn_avg_2_sigmoid])
    avg_multi_char_sigmoid = Lambda(lambda x: x[0] * x[1])([merged_pooled_rnn_avg_1_sigmoid, merged_pooled_rnn_avg_2_sigmoid])
  
    max_sub_char_abs_tanh = Lambda(lambda x:K.abs(x[0] - x[1]))([merged_pooled_rnn_max_1_tanh, merged_pooled_rnn_max_2_tanh])
    max_multi_char_tanh = Lambda(lambda x: x[0] * x[1])([merged_pooled_rnn_max_1_tanh, merged_pooled_rnn_max_2_tanh])

    avg_sub_char_abs_tanh = Lambda(lambda x:K.abs(x[0] - x[1]))([merged_pooled_rnn_avg_1_tanh, merged_pooled_rnn_avg_2_tanh])
    avg_multi_char_tanh = Lambda(lambda x: x[0] * x[1])([merged_pooled_rnn_avg_1_tanh, merged_pooled_rnn_avg_2_tanh])
  


    att_merged_pooled_rnn_max_1 = Concatenate(axis=-1)(att_pooled_rnn_max_1)
    att_merged_pooled_rnn_max_1_sigmoid = Dense(100, activation = 'sigmoid')(att_merged_pooled_rnn_max_1)
    att_merged_pooled_rnn_max_1_tanh = Dense(100, activation = 'tanh')(att_merged_pooled_rnn_max_1)

    att_merged_pooled_rnn_max_2 = Concatenate(axis=-1)(att_pooled_rnn_max_2)
    att_merged_pooled_rnn_max_2_sigmoid = Dense(100, activation = 'sigmoid')(att_merged_pooled_rnn_max_2)
    att_merged_pooled_rnn_max_2_tanh = Dense(100, activation = 'tanh')(att_merged_pooled_rnn_max_2)

    att_merged_pooled_rnn_avg_1 = Concatenate(axis=-1)(att_pooled_rnn_avg_1)
    att_merged_pooled_rnn_avg_1_sigmoid = Dense(100, activation = 'sigmoid')(att_merged_pooled_rnn_avg_1)
    att_merged_pooled_rnn_avg_1_tanh = Dense(100, activation = 'tanh')(att_merged_pooled_rnn_avg_1)
   
    att_merged_pooled_rnn_avg_2 = Concatenate(axis=-1)(att_pooled_rnn_avg_2)
    att_merged_pooled_rnn_avg_2_sigmoid = Dense(100, activation = 'sigmoid')(att_merged_pooled_rnn_avg_2)
    att_merged_pooled_rnn_avg_2_tanh = Dense(100, activation = 'tanh')(att_merged_pooled_rnn_avg_2)

    
    #sub and multi
    max_sub_char_abs_attention_sigmoid = Lambda(lambda x:K.abs(x[0] - x[1]))([att_merged_pooled_rnn_max_1_sigmoid, att_merged_pooled_rnn_max_2_sigmoid])
    max_sub_char_abs_attention_tanh = Lambda(lambda x:K.abs(x[0] - x[1]))([att_merged_pooled_rnn_max_1_tanh, att_merged_pooled_rnn_max_2_tanh])
    
    max_multi_char_attention_sigmoid = Lambda(lambda x: x[0] * x[1])([att_merged_pooled_rnn_max_1_sigmoid, att_merged_pooled_rnn_max_2_sigmoid])
    max_multi_char_attention_tanh = Lambda(lambda x: x[0] * x[1])([att_merged_pooled_rnn_max_1_tanh, att_merged_pooled_rnn_max_2_tanh])

    avg_sub_char_abs_attention_sigmoid = Lambda(lambda x:K.abs(x[0] - x[1]))([att_merged_pooled_rnn_avg_1_sigmoid, att_merged_pooled_rnn_avg_2_sigmoid])
    avg_sub_char_abs_attention_tanh = Lambda(lambda x:K.abs(x[0] - x[1]))([att_merged_pooled_rnn_avg_1_tanh, att_merged_pooled_rnn_avg_2_tanh])
    
    avg_multi_char_attention_sigmoid = Lambda(lambda x: x[0] * x[1])([att_merged_pooled_rnn_avg_1_sigmoid, att_merged_pooled_rnn_avg_2_sigmoid])
    avg_multi_char_attention_tanh = Lambda(lambda x: x[0] * x[1])([att_merged_pooled_rnn_avg_1_tanh, att_merged_pooled_rnn_avg_2_tanh])

    merged = concatenate([merged_pooled_rnn_max_1_sigmoid, merged_pooled_rnn_max_2_sigmoid, \
                          merged_pooled_rnn_avg_1_sigmoid, merged_pooled_rnn_avg_2_sigmoid, \
                          att_merged_pooled_rnn_max_1_sigmoid, att_merged_pooled_rnn_max_2_sigmoid, \
                          att_merged_pooled_rnn_avg_1_sigmoid, att_merged_pooled_rnn_avg_2_sigmoid, \
                          max_sub_char_abs_sigmoid, max_multi_char_sigmoid, avg_sub_char_abs_sigmoid, avg_multi_char_sigmoid, \
                          max_sub_char_abs_attention_sigmoid, max_multi_char_attention_sigmoid, \
                          avg_sub_char_abs_attention_sigmoid, avg_multi_char_attention_sigmoid, \
                          
                          merged_pooled_rnn_max_1_tanh, merged_pooled_rnn_max_2_tanh, \
                          merged_pooled_rnn_avg_1_tanh, merged_pooled_rnn_avg_2_tanh, \
                          att_merged_pooled_rnn_max_1_tanh, att_merged_pooled_rnn_max_2_tanh, \
                          att_merged_pooled_rnn_avg_1_tanh, att_merged_pooled_rnn_avg_2_tanh, \
                          max_sub_char_abs_tanh, max_multi_char_tanh, avg_sub_char_abs_tanh, avg_multi_char_tanh, \
                          max_sub_char_abs_attention_tanh, max_multi_char_attention_tanh, \
                          avg_sub_char_abs_attention_tanh, avg_multi_char_attention_tanh])
    
    merged = Dropout(0.5)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(512, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [16]:
model = DR_Stack_RNN()

In [18]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_Stack_att_RNN_char_v1_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = DR_Stack_RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 883s 10ms/step - loss: 1.1248 - acc: 0.6817 - val_loss: 0.9034 - val_acc: 0.7384
Epoch 2/20
92040/92040 [==============================] - 855s 9ms/step - loss: 0.8342 - acc: 0.7602 - val_loss: 0.8135 - val_acc: 0.7650
Epoch 3/20
92040/92040 [==============================] - 861s 9ms/step - loss: 0.4666 - acc: 0.8615 - val_loss: 0.8776 - val_acc: 0.7714
Epoch 7/20
102277/102277 [==============================] - 282s 3ms/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/20
92043/92043 [==============================] - 870s 9ms/step - loss: 1.1155 - acc: 0.6842 - val_loss: 0.8990 - val_acc: 0.7443
Epoch 2/20
92043/92043 [==============================] - 853s 9ms/step - loss: 0.8266 - acc: 0.7630 - val_loss: 0.8595 - val_acc: 0.7525
Epoch 3/20
92043/92043 [==============================] - 859s 9ms/step - loss: 0.7252 - acc: 0.7903 - 

In [23]:
meta_test.shape

(102277, 190)

In [24]:
pred = pd.DataFrame(test['id'])

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

pred['class'] = np.argmax(p.values, axis=1) + 1

In [26]:
pred.to_csv('/home/libo/daguan/stack_new/9.6.1.csv', index = None ,encoding = 'utf-8')

In [20]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_Stack_att_RNN_char_v1_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = DR_Stack_RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 916s 10ms/step - loss: 1.0493 - acc: 0.6998 - val_loss: 0.8728 - val_acc: 0.7432
Epoch 2/20
92040/92040 [==============================] - 895s 10ms/step - loss: 0.7944 - acc: 0.7669 - val_loss: 0.8042 - val_acc: 0.7665
Epoch 3/20
92040/92040 [==============================] - 904s 10ms/step - loss: 0.6897 - acc: 0.7940 - val_loss: 0.8143 - val_acc: 0.7682
Epoch 4/20
92040/92040 [==============================] - 903s 10ms/step - loss: 0.5933 - acc: 0.8185 - val_loss: 0.8104 - val_acc: 0.7733
Epoch 5/20
92040/92040 [==============================] - 906s 10ms/step - loss: 0.4884 - acc: 0.8489 - val_loss: 0.8304 - val_acc: 0.7704
Epoch 6/20
102277/102277 [==============================] - 302s 3ms/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/20
32256/92043 [=========>....................] - ETA: 9:38 - loss: 1.2351 - acc: 0.6526

KeyboardInterrupt: 

In [18]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_Stack_att_RNN_char_v1_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = DR_Stack_RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 919s 10ms/step - loss: 1.0332 - acc: 0.7065 - val_loss: 0.8520 - val_acc: 0.7503
Epoch 2/20
92040/92040 [==============================] - 908s 10ms/step - loss: 0.7780 - acc: 0.7713 - val_loss: 0.7937 - val_acc: 0.7658
Epoch 3/20
 1792/92040 [..............................] - ETA: 14:06 - loss: 0.6358 - acc: 0.8125

KeyboardInterrupt: 

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_Stack_att_RNN_char_v1_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = DR_Stack_RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 632s 7ms/step - loss: 1.0284 - acc: 0.7043 - val_loss: 0.8407 - val_acc: 0.7508
Epoch 2/20
92040/92040 [==============================] - 627s 7ms/step - loss: 0.7790 - acc: 0.7704 - val_loss: 0.7890 - val_acc: 0.7685
Epoch 3/20
92040/92040 [==============================] - 625s 7ms/step - loss: 0.6721 - acc: 0.7982 - val_loss: 0.7773 - val_acc: 0.7731
Epoch 4/20
92040/92040 [==============================] - 626s 7ms/step - loss: 0.5674 - acc: 0.8253 - val_loss: 0.7945 - val_acc: 0.7754
Epoch 5/20
92040/92040 [==============================] - 625s 7ms/step - loss: 0.4576 - acc: 0.8551 - val_loss: 0.8414 - val_acc: 0.7752
Epoch 6/20
102277/102277 [==============================] - 210s 2ms/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/20
92043/92043 [==============================] - 629s 7ms/step - loss: 1.0253 - acc: 0.7040 - v

92056/92056 [==============================] - 631s 7ms/step - loss: 1.0342 - acc: 0.7050 - val_loss: 0.8177 - val_acc: 0.7581
Epoch 2/20
92056/92056 [==============================] - 626s 7ms/step - loss: 0.7882 - acc: 0.7686 - val_loss: 0.7696 - val_acc: 0.7753
Epoch 3/20
77440/92057 [========================>.....] - ETA: 1:35 - loss: 1.0606 - acc: 0.6966

In [16]:
meta_train.columns = ['meta_base_Stack_att_RNN_char_300dim_v1' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [17]:
p.shape, meta_train.shape

((102277, 19), (102277, 19))

In [18]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_base_Stack_att_RNN_char_300dim_v1.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_metabase_Stack_att_RNN_char_300dim_v1.csv', index = None ,encoding = 'utf-8')

In [19]:
test = pd.read_csv('test_set.csv')

pred = pd.DataFrame(test['id'])

pred['class'] = np.argmax(p.values, axis=1) + 1

pred.to_csv('/home/libo/daguan/stack_new/9.5.1.csv', index = None ,encoding = 'utf-8')

In [20]:
pred

,id,class
0,0,5
1,1,4
2,2,13
3,3,4
4,4,5
5,5,5
6,6,15
7,7,19
8,8,3
9,9,12
